# Building ISO 8583 Messages

This notebook covers advanced message building techniques with iso8583sim.

## Setup

In [1]:
import sys

sys.path.insert(0, "..")

from iso8583sim.core.builder import ISO8583Builder
from iso8583sim.core.parser import ISO8583Parser
from iso8583sim.core.types import ISO8583Message, ISO8583Version
from iso8583sim.core.validator import ISO8583Validator

## Basic Message Building

### Creating an Authorization Request (0100)

In [2]:
builder = ISO8583Builder()

# Authorization request
auth_request = ISO8583Message(
    mti="0100",
    fields={
        0: "0100",
        2: "4111111111111111",  # PAN
        3: "000000",  # Processing code: Purchase
        4: "000000010000",  # Amount: $100.00
        11: "123456",  # STAN
        14: "2612",  # Expiry: Dec 2026
        22: "051",  # POS entry mode: chip
        23: "001",  # Card sequence number
        25: "00",  # POS condition code
        26: "12",  # PIN capture code
        32: "123456",  # Acquiring institution ID
        35: "4111111111111111=26125010000000000000",  # Track 2
        41: "TERM0001",  # Terminal ID
        42: "MERCHANT123456 ",  # Merchant ID (15 chars)
        43: "ACME Store         New York      NY US",  # Merchant name/location (40 chars)
        49: "840",  # Currency: USD
        52: "1234567890ABCDEF",  # PIN block
    },
)

raw = builder.build(auth_request)
print("Authorization Request:")
print(f"Raw: {raw[:80]}...")
print(f"Length: {len(raw)} bytes")

Authorization Request:
Raw: 0100702406C120E09000164111111111111111000000000000010000123456261205100100120612...
Length: 205 bytes


### Creating a Financial Request (0200)

In [3]:
# Financial transaction request
financial_request = ISO8583Message(
    mti="0200",
    fields={
        0: "0200",
        2: "5555555555554444",  # Mastercard test PAN
        3: "000000",  # Purchase
        4: "000000005000",  # $50.00
        11: "654321",
        12: "120000",  # Local time: 12:00:00
        13: "1225",  # Local date: Dec 25
        14: "2712",
        22: "051",
        41: "TERM0002",
        42: "STORE987654321 ",
        49: "840",
    },
)

raw = builder.build(financial_request)
print("Financial Request:")
print(f"Raw: {raw[:80]}...")
print(f"Length: {len(raw)} bytes")

Financial Request:
Raw: 0200703C040000C0800016555555555555444400000000000000500065432112000012252712051T...
Length: 105 bytes


## Creating Responses

The builder can automatically create responses from requests:

In [4]:
# Parse the request
parser = ISO8583Parser()
parsed_request = parser.parse(builder.build(auth_request))

# Create approved response
approved_response = builder.create_response(
    parsed_request,
    response_fields={
        38: "ABC123",  # Authorization code
        39: "00",  # Response code: Approved
    },
)

print("Approved Response:")
print(f"MTI: {approved_response.mti}")  # 0110
print(f"Auth Code (F38): {approved_response.fields.get(38)}")
print(f"Response Code (F39): {approved_response.fields.get(39)}")

# Verify original fields are echoed
print("\nEchoed from request:")
print(f"  PAN (F2): {approved_response.fields.get(2)}")
print(f"  Amount (F4): {approved_response.fields.get(4)}")
print(f"  STAN (F11): {approved_response.fields.get(11)}")

Approved Response:
MTI: 0110
Auth Code (F38): ABC123
Response Code (F39): 00

Echoed from request:
  PAN (F2): 4111111111111111
  Amount (F4): 000000010000
  STAN (F11): 123456


In [5]:
# Create declined response
declined_response = builder.create_response(
    parsed_request,
    response_fields={
        39: "51",  # Insufficient funds
    },
)

print("Declined Response:")
print(f"MTI: {declined_response.mti}")
print(f"Response Code (F39): {declined_response.fields.get(39)} - Insufficient Funds")

Declined Response:
MTI: 0110
Response Code (F39): 51 - Insufficient Funds


## Processing Codes (Field 3)

The processing code defines the transaction type:

In [6]:
processing_codes = {
    "000000": "Purchase",
    "010000": "Cash Withdrawal",
    "090000": "Purchase with Cashback",
    "200000": "Refund",
    "280000": "Payment",
    "300000": "Balance Inquiry",
    "310000": "Mini Statement",
}

print("Processing Code Examples:")
print("-" * 40)
for code, description in processing_codes.items():
    print(f"{code}: {description}")

# Build a refund transaction
refund = ISO8583Message(
    mti="0200",
    fields={
        0: "0200",
        2: "4111111111111111",
        3: "200000",  # Refund
        4: "000000002500",  # $25.00 refund
        11: "111111",
        41: "TERM0001",
        42: "MERCHANT123456 ",
    },
)

print("\nRefund Transaction Built:")
print(f"Processing Code: {refund.fields[3]}")

Processing Code Examples:
----------------------------------------
000000: Purchase
010000: Cash Withdrawal
090000: Purchase with Cashback
200000: Refund
280000: Payment
300000: Balance Inquiry
310000: Mini Statement

Refund Transaction Built:
Processing Code: 200000


## Reversal Messages (0400)

Reversals are used to cancel or void transactions:

In [7]:
# Original transaction
original = ISO8583Message(
    mti="0200",
    fields={
        0: "0200",
        2: "4111111111111111",
        3: "000000",
        4: "000000010000",
        11: "999888",
        12: "143022",
        13: "1215",
        37: "123456789012",  # RRN
        38: "XYZ789",  # Auth code
        41: "TERM0001",
        42: "MERCHANT123456 ",
    },
)

# Create reversal
reversal = ISO8583Message(
    mti="0400",
    fields={
        0: "0400",
        2: original.fields[2],  # Same PAN
        3: original.fields[3],  # Same processing code
        4: original.fields[4],  # Same amount
        11: "999889",  # New STAN
        12: original.fields[12],
        13: original.fields[13],
        37: original.fields[37],  # Original RRN
        38: original.fields[38],  # Original auth code
        41: original.fields[41],
        42: original.fields[42],
        90: f"0200{original.fields[11]}1215143022000000000000000000",  # Original data element
    },
)

raw_reversal = builder.build(reversal)
print("Reversal Message:")
print(f"MTI: {reversal.mti}")
print(f"Field 90 (Original Data): {reversal.fields[90]}")
print(f"\nRaw: {raw_reversal[:60]}...")

Reversal Message:
MTI: 0400
Field 90 (Original Data): 000002009998881215143022000000000000000000

Raw: 0400F03800000CC000000000004000000000164111111111111111000000...


## Network Management Messages (0800/0810)

Used for sign-on, sign-off, echo test, and key exchange:

In [8]:
# Echo test request
echo_request = ISO8583Message(
    mti="0800",
    fields={
        0: "0800",
        7: "1215143022",  # Transmission date/time
        11: "000001",  # STAN
        70: "301",  # Network management info code: Echo test
    },
)

raw_echo = builder.build(echo_request)
print("Echo Test Request:")
print(f"Raw: {raw_echo}")

# Sign-on request
signon = ISO8583Message(
    mti="0800",
    fields={
        0: "0800",
        7: "1215143022",
        11: "000002",
        70: "001",  # Sign-on
    },
)

print("\nSign-on Request:")
print(f"Raw: {builder.build(signon)}")

Echo Test Request:
Raw: 0800822000000000000004000000000000001215143022000001301

Sign-on Request:
Raw: 0800822000000000000004000000000000001215143022000002001


## Field Validation During Build

The builder validates fields as they're processed:

In [9]:
validator = ISO8583Validator()

# Create a message with potential issues
test_msg = ISO8583Message(
    mti="0100",
    fields={
        0: "0100",
        2: "4111111111111111",
        3: "000000",
        4: "000000001000",
        11: "123456",
        41: "TERM0001",
        42: "MERCHANT123456 ",
    },
)

# Build and validate
raw = builder.build(test_msg)
parsed = parser.parse(raw)
errors = validator.validate_message(parsed)

print(f"Validation Result: {'PASS' if not errors else 'FAIL'}")
if errors:
    print("Errors:")
    for e in errors:
        print(f"  - {e}")

Validation Result: FAIL
Errors:
  - Required field 14 missing for VISA
  - Required field 22 missing for VISA
  - Required field 24 missing for VISA
  - Required field 25 missing for VISA


## Building with Different Versions

In [10]:
# ISO 8583:1987 (most common)
msg_1987 = ISO8583Message(mti="0100", version=ISO8583Version.V1987, fields={0: "0100", 3: "000000", 11: "123456"})

# ISO 8583:1993
msg_1993 = ISO8583Message(
    mti="1100",  # Note: version digit is '1'
    version=ISO8583Version.V1993,
    fields={0: "1100", 3: "000000", 11: "123456"},
)

print("Messages by Version:")
print(f"1987: {builder.build(msg_1987)}")
print(f"1993: {builder.build(msg_1993)}")
print("\nNote: ISO 8583:2003 uses MTI starting with '2' but has different validation rules.")

Messages by Version:
1987: 01002020000000000000000000123456
1993: 11002020000000000000000000123456

Note: ISO 8583:2003 uses MTI starting with '2' but has different validation rules.


## Performance: Batch Building

In [11]:
import time

# Reuse builder instance for performance
builder = ISO8583Builder()

start = time.perf_counter()
messages = []
for i in range(1000):
    msg = ISO8583Message(
        mti="0100",
        fields={
            0: "0100",
            2: f"411111111111{i:04d}",
            3: "000000",
            4: f"{(i * 100):012d}",
            11: f"{i:06d}",
            41: "TERM0001",
            42: "MERCHANT123456 ",
        },
    )
    messages.append(builder.build(msg))
elapsed = time.perf_counter() - start

print(f"Built {len(messages)} messages in {elapsed:.3f}s")
print(f"Throughput: {len(messages)/elapsed:,.0f} messages/second")

Built 1000 messages in 0.007s
Throughput: 133,714 messages/second


## Next Steps

- **[04_network_specifics.ipynb](04_network_specifics.ipynb)** - VISA, Mastercard specifics
- **[05_emv_data.ipynb](05_emv_data.ipynb)** - Building messages with EMV data